<a href="https://colab.research.google.com/github/PaulOnKithub/MalariaCellProject/blob/master/MalariaCellRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


K = 100
for layer in base_model.layers[:K]:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global Average Pooling to reduce spatial dimensions
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification (malaria-infected or not)


Malaria_model = Model(inputs=base_model.input, outputs=predictions)


Malaria_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Malaria_model.summary()


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)


path_to_train_folder = '/content/drive/MyDrive/kaggle/cell_images/Train'
path_to_validation_folder = '/content/drive/MyDrive/kaggle/cell_images/Validate'



train_data = train_datagen.flow_from_directory(
    path_to_train_folder,
    target_size=(224, 224),
    batch_size=50,
    class_mode='binary'
)

validation_data = validation_datagen.flow_from_directory(
    path_to_validation_folder,
    target_size=(224, 224),
    batch_size=50,
    class_mode='binary'
)


In [ ]:
num_epochs = 5

history = Malaria_model.fit(
    train_data,
    epochs=num_epochs,
    validation_data=validation_data
)

Epoch 1/5
113/541 [=====>........................] - ETA: 1:53:07 - loss: 0.1609 - accuracy: 0.9416

In [ ]:
evaluation = Malaria_model.evaluate(validation_data)
print("Validation Loss:", evaluation[0])
print("Validation Accuracy:", evaluation[1])

11/11 [==============================] - 68s 6s/step - loss: 0.2143 - accuracy: 0.9359
Validation Loss: 0.21430881321430206
Validation Accuracy: 0.9359223246574402


In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation loss
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

plt.tight_layout()
plt.show()


In [ ]:
# Saving the trained model
Malaria_model.save('/content/drive/MyDrive/kaggle/trained_malaria_model.h5')


In [ ]:
!pip install tf2onnx

In [4]:
import tensorflow as tf

model_path = '/content/drive/MyDrive/kaggle/trained_malaria_model.h5'
model = tf.keras.models.load_model(model_path)


In [5]:
import tf2onnx
onnx_model, _ = tf2onnx.convert.from_keras(model)

In [8]:
# Saving the ONNX model to Google Drive
onnx_model_path = '/content/drive/MyDrive/kaggle/trained_malaria_model.onnx'
with open(onnx_model_path, 'wb') as f:
    f.write(onnx_model.SerializeToString())


In [ ]:
!pip install onnxruntime


In [21]:
import cv2
import numpy as np
import onnxruntime

# Creating an ONNX Runtime inference session
ort_session = onnxruntime.InferenceSession(onnx_model_path)

# Loading and preprocessing a test image
image_path = '/content/drive/MyDrive/kaggle/test_image.png'
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
image = cv2.resize(image, (224, 224))  # Resize to match the model's input size
image = image.astype(np.float32) / 255.0  # Normalize pixel values

# Preparing the input tensor
input_name = ort_session.get_inputs()[0].name
input_data = np.expand_dims(image, axis=0)

# Running inference
output = ort_session.run(None, {input_name: input_data})

# finding the class with the highest probability
predicted_class_index = np.argmax(output)

class_labels = {
    0: "Parasitized",
    1: "Uninfected"
}

predicted_class_label = class_labels.get(predicted_class_index, "Unknown")

# Print the predicted class label
print("Predicted Class:", predicted_class_label)


Predicted Class: Parasitized


In [25]:
import tensorflow as tf
import cv2
import numpy as np

# Loading the Keras model from the H5 file
model = tf.keras.models.load_model('/content/drive/MyDrive/kaggle/trained_malaria_model.h5')

# Loading and preprocessing a test image
image_path = '/content/drive/MyDrive/kaggle/test_image4.png'
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
image = cv2.resize(image, (224, 224))  # Resize to match the model's input size
image = image.astype(np.float32) / 255.0  # Normalize pixel values

# Preparing the input tensor
input_data = np.expand_dims(image, axis=0)

# Running inference
predictions = model.predict(input_data)

# finding the class with the highest probability
predicted_class = np.argmax(predictions)

# Printing the predicted class
print("Predicted Class:", predicted_class)


1/1 [==============================] - 2s 2s/step
Predicted Class: 0
